# 국민 청원 크롤링
* URL : https://www1.president.go.kr/petitions

In [2]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
import numpy as np
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러올 수 있습니다.
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용합니다.
from bs4 import BeautifulSoup as bs
# 랜덤숫자를 생성한다.
import random
import time
# 대량 데이터 처리시 진행 상황을 표시합니다.
from tqdm import tqdm, trange
# 정규표현식
import re

In [3]:
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

## 국민청원 top 5 크롤링

In [5]:
# suggestion_list=[]
# base_url ='https://www1.president.go.kr/petitions/?c=0&only=1&page=2&order=0'
# response = requests.get( base_url )
# html = bs(response.text, 'html.parser')
# # 소스코드를 제대로 가져왔는지 확인해 봅니다.
# tags=html.select('#cont_view > div.cs_area > div > div > div.ct_petitions > div.ct_petitions_area.ct_txt_left > div.ct_list > div > div > div.bl_body > ul ')[0].find_all('a')
# # print(tags)
# for tag in tags:
#     suggestion_list.append(tag['href'])

In [6]:
suggestion_list=[]
base_url ='https://www1.president.go.kr/petitions/?c=0&only=1&page=2&order=0'

driver =webdriver.Chrome()
driver.get(base_url)
html=driver.page_source
soup=BeautifulSoup(html)

# 소스코드를 제대로 가져왔는지 확인해 봅니다.
tags=soup.select('#cont_view > div.cs_area > div > div > div.ct_petitions > div.ct_petitions_area.ct_txt_left > div.ct_list > div > div > div.bl_body > ul ')[0].find_all('a')
# print(tags)
for tag in tags:
    suggestion_list.append(tag['href'])

driver.close()

In [4]:
print(suggestion_list)

['/petitions/584936', '/petitions/585290', '/petitions/585683', '/petitions/585007', '/petitions/584755']


In [5]:
def contents_func(url):
    article = []
    baseurl=f'https://www1.president.go.kr{url}'
    driver =webdriver.Chrome()
    driver.get(baseurl)
    
    html=driver.page_source
    soup=BeautifulSoup(html)

    title=soup.select('#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > h3')[0].get_text(strip=True)

    desc = soup.select(
                '#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul')
    desc1=desc[0].get_text(strip=False)

    category=desc1.split('\n')[1][4:]
    start=desc1.split('\n')[2][4:]
    end=desc1.split('\n')[3][4:]
    author=desc1.split('\n')[4][3:]

    content=soup.select('#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_write > div.View_write')[0].get_text(strip=True)
    vote = soup.select('#petitionsReply_area > div.Reply_area_head > h3 > span')[0].get_text(strip=True)
    
    article.append(title)
    article.append(category)
    article.append(content)
    article.append(start)
    article.append(end)
    article.append(vote)
    article.append(author)

    driver.close()
    return article


In [6]:
data = []
# tqdm 을 사용해서 어느정도 가져왔는지 확인하기
for i, suggestion in tqdm(enumerate(suggestion_list)):
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져옵니다.
    s = contents_func(suggestion)
    data.append(s)

5it [01:16, 15.27s/it]


In [7]:
column_names = ["title", "category", "content", "start", "end", "vote", "author"]
column_names

data_top5 = pd.DataFrame(data, columns=column_names)
data_top5.head()

,title,category,content,start,end,vote,author
0,문재인 대통령 탄핵을 촉구합니다.,정치개혁,국민의 한 사람으로서 문재인 대통령의 탄핵을 촉구합니다.이번 우한 폐렴(신종 코로나...,2020-02-04,2020-03-05,"849,845",naver - ***
1,"신천지 예수교 증거장막성전(이하, 신천지)의 강제 해체(해산)을 청원합니다.",인권/성평등,"신천지 예수교 증거장막성전(이하, 신천지)의 강제 해체(해산)을 청원합니다.“MBC...",2020-02-22,2020-03-23,"837,686",facebook - ***
2,문재인 대통령님을 응원 합니다!,기타,코로나 바이러스19로 인해 대한민국 모든 국민이 힘든 시기에 있습니다. 하지만 국민...,2020-02-26,2020-03-27,"285,718",naver - ***
3,저의 아들이 6년간다닌어린이집 에서 남자원장으로부터 3년간 성폭행을 당했습니다,육아/교육,저의 아들은 7살입니다.생후 25일일때부터 입소하여서 7살인2020년1월18일까지 ...,2020-02-07,2020-03-08,"184,801",naver - ***
4,추미애 법무장관의 해임을 청원합니다,정치개혁,[ 추미애 법무장관의 해임을 청원합니다. ]우리 검찰의 역사는 정권의 수족역할을 해...,2020-02-03,2020-03-04,"183,063",naver - ***


In [25]:
data_top5['start']=pd.to_datetime(data_top5['start'])
data_top5['end']=pd.to_datetime(data_top5['end'])

In [40]:
data_top5['vote']=data_top5['vote'].str.replace(',','')

In [42]:
data_top5['vote']=data_top5['vote'].astype('int')

## 전체 국민 청원 크롤링

In [154]:
suggestion_list=[]
for i in range(1):
    pnum=i
    base_url=f'https://www1.president.go.kr/petitions/?c=0&only=1&page={pnum+1}&order=0'

    driver =webdriver.Chrome()
    driver.get(base_url)

    html=driver.page_source
    soup=BeautifulSoup(html)

    tags=soup.select('#cont_view > div.cs_area > div > div > div.ct_petitions > div.ct_petitions_area.ct_txt_left > div.ct_list1 > div > div.b_list.category.b_list2 > div.bl_body > ul ')[0].find_all('a')
    for tag in tags:
            suggestion_list.append(tag['href'])

In [156]:
def contents_func(url):
    article = []
    baseurl=f'https://www1.president.go.kr{url}'
    driver =webdriver.Chrome()
    driver.get(baseurl)
    driver.close()
    html=driver.page_source
    soup=BeautifulSoup(html)

    title=soup.select('#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > h3')[0].get_text(strip=True)

    desc = soup.select(
                '#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_info > ul')
    desc1=desc[0].get_text(strip=False)

    category=desc1.split('\n')[1][4:]
    start=desc1.split('\n')[2][4:]
    end=desc1.split('\n')[3][4:]
    author=desc1.split('\n')[4][3:]

    content=soup.select('#cont_view > div.cs_area > div.new_contents > div > div.petitionsView_left > div > div.petitionsView_write > div.View_write')[0].get_text(strip=True)
    vote = soup.select('#petitionsReply_area > div.Reply_area_head > h3 > span')[0].get_text(strip=True)
    article.append(title)
    article.append(category)
    article.append(content)
    article.append(start)
    article.append(end)
    article.append(vote)
    article.append(author)

    return article


In [158]:
data = []
# tqdm 을 사용해서 어느정도 가져왔는지 확인하기
for i, suggestion in tqdm(enumerate(suggestion_list)):
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져옵니다.
    s = contents_func(suggestion)
    data.append(s)

30it [06:02, 12.08s/it]


In [159]:
column_names = ["title", "category", "content", "start", "end", "vote", "author"]
column_names

data = pd.DataFrame(data, columns=column_names)
data.head()

,title,category,content,start,end,vote,author
0,판매용이 아닌 기업 자체 소비용 보건용 마스크 수입요건확인 면제요청 (긴급),보건복지,기업자체 사용용도 보건용 마스크 수입요건확인 면제(긴급)청와대국민청원을 긴급하게 올...,2020-02-26,2020-03-27,195,kakao - ***
1,(주)***의 부당한 서비스 이용 제한 및 강제적인 개인 휴대전화번호 변경 요구,인권/성평등,너무나도 부당한 (주)*** 측의 처사를 참을 수가 없어서 국민 청원 글을 올립니다...,2020-02-26,2020-03-27,110,naver - ***
2,건축사 시험 연기 를 청원 합니다,교통/건축/국토,건축사 시험일이 3월 21일입니다 전국 각지 에서 모여 서 9시간동안 한교실...,2020-02-26,2020-03-27,302,naver - ***
3,"2020년 아이스하키 U18 남자 대표선수 훈련 일정을 중단하고,해외에서 열리는 대...",문화/예술/체육/언론,"현재 21일 0시를 기준으로 중국,일본,대한민국 뿐만 아니라 전세계가 코로나19 감...",2020-02-26,2020-03-27,129,naver - ***
4,청년내일채움공제 사회복지사도 신청하게 해주세요 ㅠㅠ,보건복지,사회복지관련 센터에서 일을 하면 취직하고 월급이.. 세금 제외시키고 200도 안되요...,2020-02-26,2020-03-27,178,naver - ***


In [ ]:
#### ===============